In [31]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

In [32]:
%%time

ep = 1e-4
time = np.arange(0,10,ep)
L = 1*np.float32(np.logical_and(time>2,time<4))

kr = 1/np.random.uniform(0,0.6)
kf = 1/np.random.uniform(0,1.2)

Cm = 0.00144
gL = 1.44
Vl = -62
VR = 0

gR = 0.01#0.25
thresh0 = -55
reset = -62

tGamma = 1.2
delth = np.random.uniform(0.05,0.1)#np.random.uniform

gB = 0.19
gN = 0.0

def f(y,t):
    Rs,V = y
    dRs = kr*L[int(t/ep)]*(1-Rs)-kf*Rs
    dV = 1/Cm*(-gL*(V-Vl)-gR*(V-VR)*Rs -gB*(V-VR)- gN*np.random.normal()*(V-VR))
    return np.array([dRs,dV])

X = np.zeros((2,time.shape[0]))
wth = np.zeros(time.shape[0])
sp = np.zeros(time.shape[0])

X[:,0]= [0,-60]

for i in range(1,time.shape[0]):
    X[:,i] = X[:,i-1] + ep*f(X[:,i-1],time[i-1])
    wth[i] = wth[i-1]*np.exp(-ep/tGamma) 
    if X[1,i]>thresh0+wth[i]:
        X[1,i] = reset
        sp[i-1] = 1
        wth[i] = wth[i] + delth/tGamma

plt.figure(figsize=(15,3))
plt.plot(time[:],X[1,:],"-",label="RK4 Solution for y")
plt.scatter(time[sp>0],sp[sp>0]+thresh0,marker="|",s=10000)

plt.xlabel("t")
plt.ylabel("X")
plt.ylim([-100,0])
plt.legend()
plt.show()


plt.figure(figsize=(15,3))

sp_times = np.arange(time.shape[0])[np.bool8(sp)]

plt.plot(sp_times[1:-1]*ep,1/np.diff(sp_times*ep)[1:],"-",label="RK4 Solution for y")
#plt.scatter(time[sp>0],sp[sp>0]+thresh0,marker="|",s=10000)

plt.xlabel("t")
plt.ylabel("X")
plt.ylim([0,20])
plt.xlim([0,10])
plt.legend()
plt.show()




CPU times: user 4.17 s, sys: 24 ms, total: 4.19 s
Wall time: 4.17 s


In [33]:
plt.plot(wth[::100])

In [85]:
%%time
n = 20
ep = 1e-4
time = np.arange(0,10,ep)
L = 1*np.float32(np.logical_and(time>2,time<4))

kr = 1/np.random.uniform(0,0.6,size=n)
kf = 1/np.random.uniform(0,1.2,size=n)

Cm = 0.00144
gL = 1.44
Vl = -62
VR = 0

gR = 0.01+0.01*np.random.normal(size=n)#0.25
thresh0 = -55
reset = -62

tGamma = np.random.uniform(0.1,1.2,size=n)
delth = np.random.uniform(0.05,0.1,size=n)#np.random.uniform

gB = 0.19
gN = 0.005

def f(y,t):
    Rs,V = y[:n],y[n:]
    dRs = kr*L[int(t/ep)]*(1-Rs)-kf*Rs
    dV = 1/Cm*(-gL*(V-Vl)-gR*(V-VR)*Rs -gB*(V-VR)- gN*np.random.normal()*(V-VR))
    return np.concatenate([dRs,dV])

X = np.zeros((2*n,time.shape[0]))
wth = np.zeros((n,time.shape[0]))
sp = np.zeros((n,time.shape[0]))

X[:,0]= [0]*n+[-60]*n

for i in range(1,time.shape[0]):
    X[:,i] = X[:,i-1] + ep*f(X[:,i-1],time[i-1])
    wth[:,i] = wth[:,i-1]*np.exp(-ep/tGamma)
    #if X[1,i]>thresh0+wth[i]:
    cond = X[n:,i]>wth[:,i]+thresh0
    X[n:,i] = np.where(cond,reset*np.ones(n),X[n:,i])
    sp[:,i] = np.where(cond,np.ones(n),np.zeros(n))
    wth[:,i] = np.where(cond,wth[:,i] + delth/tGamma,wth[:,i])



CPU times: user 8.83 s, sys: 64 ms, total: 8.89 s
Wall time: 8.82 s


In [86]:

plt.figure(figsize=(15,3))
for i in range(n):
    plt.plot(time[::10],100*i+X[n+i,::10],"-")
    plt.scatter(time[sp[i,:]>0],100*i+sp[i,:][sp[i,:]>0]+thresh0,marker="|",s=100)

plt.xlabel("t")
plt.ylabel("X")
#plt.ylim([-100,0])
plt.show()


plt.figure(figsize=(15,3))
for i in range(n):
    sp_times = np.arange(time.shape[0])[np.bool8(sp[i,:])]
    plt.plot(sp_times[1:-1]*ep,1/np.diff(sp_times*ep)[1:],"-")
#plt.scatter(time[sp>0],sp[sp>0]+thresh0,marker="|",s=10000)

# plt.xlabel("t")
# plt.ylabel("X")
# plt.ylim([0,100])""
# plt.xlim([0,10])
# plt.legend()
# plt.show()